In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import re
from sklearn.metrics import roc_auc_score

In [3]:
dftest = pd.read_csv('all_pred_mmg_bdms_backtest.csv')

In [4]:
dftest = dftest.sort_values(by = 'Image Index')
CVAT = []
for i in range(len(dftest)):
    CVAT.append(i)
dftest['CVATPAGE']=CVAT 
dftest.head()
    

,Image Index,Cancer Prediction,Calcification Prediction,Mass Prediction,Cancer,Calcification,Mass,CVATPAGE
83,0020084008,0.362040,0.362040,0.020477,-2,-2,-2,0
485,1181027511,0.798081,0.037921,0.798081,-2,-2,-2,1
550,119029418,0.023993,0.023993,0.005868,-2,-2,-2,2
74,190110SDPMG030,0.018763,0.018763,0.005201,-2,-2,-2,3
581,19822953-636,0.985621,0.159290,0.985621,-2,-2,-2,4


In [5]:
dfgt = pd.read_csv('GT.csv')

In [6]:
dfgt.head()

,Unnamed: 0.1,Unnamed: 0,Reports,ACC No.,BIRADS,Mmg Calcification,Mmg Mass,Disease Name,All_calcification,Suspicious_calcification,Mass
0,0,0,Clinical: Routine annual mammog...,28412120-45,BIRADS2,1.0,1.0,alcification|ass,1.0,0.0,0.0
1,1,1,CLINICAL: Screening\nwith no presenting sympt...,28408002-138,BIRADS3,1.0,1.0,alcification|ass,1.0,0.0,0.0
2,2,2,"CLINICAL: palpable mass left 2-3 o'clock,post ...",28413118-47,BIRADS6,1.0,1.0,alcification|ass,1.0,1.0,1.0
3,3,3,Clinical information: Screening \n \nBILATERA...,28905280-118,BIRADS4B,1.0,NaN,alcification,1.0,0.0,0.0
4,4,4,CLINICAL: Screening\nBILATERAL DIGITAL MAMMOGR...,28883412-47,BIRADS2,1.0,1.0,alcification|ass,1.0,0.0,0.0


In [7]:
dftest.head()

,Image Index,Cancer Prediction,Calcification Prediction,Mass Prediction,Cancer,Calcification,Mass,CVATPAGE
83,0020084008,0.362040,0.362040,0.020477,-2,-2,-2,0
485,1181027511,0.798081,0.037921,0.798081,-2,-2,-2,1
550,119029418,0.023993,0.023993,0.005868,-2,-2,-2,2
74,190110SDPMG030,0.018763,0.018763,0.005201,-2,-2,-2,3
581,19822953-636,0.985621,0.159290,0.985621,-2,-2,-2,4


In [8]:
dftest = dftest.drop('Mass',axis = 1)

In [9]:
dfgt[dfgt['ACC No.'] == dftest['Image Index'].iloc[7]]

,Unnamed: 0.1,Unnamed: 0,Reports,ACC No.,BIRADS,Mmg Calcification,Mmg Mass,Disease Name,All_calcification,Suspicious_calcification,Mass


In [10]:
dfgt = dfgt.rename(columns = {'ACC No.':'Image Index'})

In [11]:
dfgt = dfgt[['Image Index','Reports','Mass','Suspicious_calcification','All_calcification','BIRADS']]
dfgt.head()

,Image Index,Reports,Mass,Suspicious_calcification,All_calcification,BIRADS
0,28412120-45,Clinical: Routine annual mammog...,0.0,0.0,1.0,BIRADS2
1,28408002-138,CLINICAL: Screening\nwith no presenting sympt...,0.0,0.0,1.0,BIRADS3
2,28413118-47,"CLINICAL: palpable mass left 2-3 o'clock,post ...",1.0,1.0,1.0,BIRADS6
3,28905280-118,Clinical information: Screening \n \nBILATERA...,0.0,0.0,1.0,BIRADS4B
4,28883412-47,CLINICAL: Screening\nBILATERAL DIGITAL MAMMOGR...,0.0,0.0,1.0,BIRADS2


In [12]:
dfgt[dfgt['Image Index']=='100749-20130116']

,Image Index,Reports,Mass,Suspicious_calcification,All_calcification,BIRADS


In [13]:
'9190110SDPMG030' in dfgt['Image Index'].values

False

In [14]:
index = []
GTindex = dfgt['Image Index'].values

In [15]:
df = pd.merge(dfgt,dftest,how = 'inner',on = 'Image Index')

In [16]:
tBIRAD = []
for i in df['BIRADS']:
    tBIRAD.append('BIRADS' +' '+ re.findall('.*(\d)',i )[0])
df['tBIRAD'] = tBIRAD

In [17]:
df = df.drop_duplicates(subset=['Image Index'])

In [18]:
len(df)

584

In [19]:
Threshold = 0.488867

In [20]:
df['Mass'] = df.Mass.astype(str)

In [21]:
mdf = df[['Reports','Mass','Mass Prediction']]
mdf = mdf[(mdf['Mass']=='1.0')|(mdf['Mass']=='0.0')]
mdf = mdf.dropna()
mTP = len(df[(df['Mass Prediction'] >= Threshold)&(df['Mass']=='1.0')])
mTN = len(df[(df['Mass Prediction'] < Threshold)&(df['Mass']=='0.0')])   
mFP = len(df[(df['Mass Prediction'] >= Threshold)&(df['Mass']=='0.0')])
mFN = len(df[(df['Mass Prediction'] < Threshold)&(df['Mass']=='1.0')])
msense = mTP/(mTP+mFN)
mspec = mTN/(mTN+mFP)
mAUC = roc_auc_score(mdf['Mass'],mdf['Mass Prediction'])

In [22]:
msense

0.7096774193548387

In [23]:
mFN

9

In [24]:
df[df['CVATPAGE']==30]

,Image Index,Reports,Mass,Suspicious_calcification,All_calcification,BIRADS,Cancer Prediction,Calcification Prediction,Mass Prediction,Cancer,Calcification,CVATPAGE,tBIRAD
176,22503089-45,CLINICAL:Follow up benign findind with during ...,0.0,0.0,1.0,BIRADS2,0.996437,0.996437,0.121744,-2,-2,30,BIRADS 2


In [27]:
dftest[dftest['CVATPAGE']==292]

,Image Index,Cancer Prediction,Calcification Prediction,Mass Prediction,Cancer,Calcification,CVATPAGE
649,28455516-41,0.975853,0.975853,0.796847,-2,-2,292


In [28]:
df[df['CVATPAGE']==81]['tBIRAD']

163    BIRADS 3
Name: tBIRAD, dtype: object

In [29]:
cdf = df[['Reports','Suspicious_calcification','Calcification Prediction']]
cdf = cdf[(cdf['Suspicious_calcification']=='1.0')|(cdf['Suspicious_calcification']=='0.0')]
cdf = cdf.dropna()
cTP = len(df[(df['Calcification Prediction'] >= Threshold)&(df['Suspicious_calcification']=='1.0')])
cTN = len(df[(df['Calcification Prediction'] < Threshold)&(df['Suspicious_calcification']=='0.0')])   
cFP = len(df[(df['Calcification Prediction'] >= Threshold)&(df['Suspicious_calcification']=='0.0')])
cFN = len(df[(df['Calcification Prediction'] < Threshold)&(df['Suspicious_calcification']=='1.0')])
csense = cTP/(cTP+cFN)
cspec = cTN/(cTN+cFP)
cAUC = roc_auc_score(cdf['Suspicious_calcification'],cdf['Calcification Prediction'])

In [30]:
for i in df[(df['Calcification Prediction'] >= Threshold)&(df['Suspicious_calcification']==0.0)].Reports:
    print(i)

In [31]:
msense

0.7096774193548387

In [32]:
mspec

0.835016835016835

In [33]:
mAUC

0.7750624524818073

In [34]:
csense

0.9523809523809523

In [35]:
cspec

0.42249240121580545

In [36]:
cAUC

0.6665219279201042

In [37]:
acdf = df[['Reports','All_calcification','Calcification Prediction']]
acdf = acdf[(acdf['All_calcification']=='1.0')|(acdf['All_calcification']=='0.0')]
acdf = acdf.dropna()
acTP = len(df[(df['Calcification Prediction'] >= Threshold)&(df['All_calcification']=='1.0')])
acTN = len(df[(df['Calcification Prediction'] < Threshold)&(df['All_calcification']=='0.0')])   
acFP = len(df[(df['Calcification Prediction'] >= Threshold)&(df['All_calcification']=='0.0')])
acFN = len(df[(df['Calcification Prediction'] < Threshold)&(df['All_calcification']=='1.0')])
acsense = acTP/(acTP+acFN)
acspec = acTN/(acTN+acFP)
acAUC = roc_auc_score(acdf['All_calcification'],acdf['Calcification Prediction'])

In [38]:
re.findall('.*(\d)','BIRADS2' )[0]

'2'

In [39]:
# BIRADS in All FP calcification cases 
df[(df['Calcification Prediction'] >= Threshold)&(df['Suspicious_calcification']=='0.0')]['tBIRAD'].value_counts()

BIRADS 2    118
BIRADS 3     54
BIRADS 4     12
BIRADS 1      5
BIRADS 6      1
Name: tBIRAD, dtype: int64

In [40]:
mAUC

0.7750624524818073

In [41]:
248/257

0.9649805447470817

In [42]:
df[(df['Mass']!='1.0')&(df['Mass']!='0.0')&(df['Mass']!='Tomosynthesis')][['Suspicious_calcification','Calcification Prediction']]
                        

,Suspicious_calcification,Calcification Prediction
122,0.0,0.213830
124,0.0,0.830943
131,0.0,0.987111
413,1.0,0.997158
415,0.0,0.017356
416,0.0,0.150196
419,0.0,0.038155
437,0.0,0.634656
441,0.0,0.042223
444,0.0,0.023333


In [43]:
mAUC

0.7750624524818073

In [44]:
cTP+cFP

210

In [45]:
cFN

1

In [46]:
131+78+22+6+4

241

In [47]:
df[df['Mass']=='nan']

,Image Index,Reports,Mass,Suspicious_calcification,All_calcification,BIRADS,Cancer Prediction,Calcification Prediction,Mass Prediction,Cancer,Calcification,CVATPAGE,tBIRAD
122,28461600-56,"CLINICAL: Left MRM, sent to follow up\nP.E.: ...",nan,0.0,0.0,BIRADS3,0.213830,0.213830,0.051720,-2,-2,311,BIRADS 3
124,28114856-105,CLINICAL:Follow up probably benign lesion\n W...,nan,0.0,1.0,BIRADS3,0.830943,0.830943,0.504317,-2,-2,132,BIRADS 3
131,24574305-82,CLINICAL: Follow up study\nComparison study: ...,nan,0.0,1.0,BIRADS3,0.987111,0.987111,0.074689,-2,-2,51,BIRADS 3
413,28544181-353,\nPROCEDURE:MAMMOGRAMS WITH ULTRASOUND BREAST...,nan,1.0,1.0,BIRADS3,0.997158,0.997158,0.117895,-2,-2,464,BIRADS 3
415,28115358-184,\nPROCEDURE:MAMMOGRAMS WITH ULTRASOUND BREAST...,nan,0.0,1.0,BIRADS2,0.095704,0.017356,0.095704,-2,-2,133,BIRADS 2
416,28529268-158,\nPROCEDURE:MAMMOGRAMS WITH ULTRASOUND BREAST...,nan,0.0,1.0,BIRADS2,0.257760,0.150196,0.257760,-2,-2,433,BIRADS 2
419,28055716-776,\nPROCEDURE:MAMMOGRAMS WITH ULTRASOUND BREAST...,nan,0.0,1.0,BIRADS2,0.485211,0.038155,0.485211,-2,-2,117,BIRADS 2
437,28444991-52,\nPROCEDURE:MAMMOGRAMS WITH ULTRASOUND BREAST...,nan,0.0,1.0,BIRADS3,0.634656,0.634656,0.019520,-2,-2,274,BIRADS 3
441,28370838-174,\nPROCEDURE:MAMMOGRAMS WITH ULTRASOUND BREAST...,nan,0.0,1.0,BIRADS2,0.042223,0.042223,0.008812,-2,-2,203,BIRADS 2
444,28557720-458,\nPROCEDURE:MAMMOGRAMS WITH ULTRASOUND BREAST...,nan,0.0,1.0,BIRADS2,0.023333,0.023333,0.005511,-2,-2,486,BIRADS 2


In [48]:
print(df[df['CVATPAGE'] == 548]['Reports'].iloc[0])

CLINICAL:  Palpable left axillary region
BILATERAL DIGITAL MAMMOGRAPHY:
Technique:standard CC, MLO views
Comparison study:   This is a baseline study.                           
Breast tissue  composition  Heterogeneously dense fibroglandular tissue with could obscure detection of small masses.
Mass    Right breast:  - A irregular hyperdense mass in right outer mid part , highly suggestive of malignancy 
                                  - Also noted a 23.6x5.3 mm  suspicious grouping pleomorphic microcalcifications in ductal distribution at  right upper outer quadrant, just posterolateral aspect to a suspicious mass about 22.5 mm,highly suggestive of malignancy
            Left breast: - An obscured isodense mass at left lower mid part, better seen on the ultrasound that  low suspicious abnormality
Macrocalcification  Not detectable
Architectural distortion Not detectable
Focal asymmetry/Focal density  Not detectable
Dilated tubular structure  Not detectable
Skin thickening           

In [52]:
df[(df['Mass Prediction']>=Threshold)&(df['Mass']=='1.0')].head()

,Image Index,Reports,Mass,Suspicious_calcification,All_calcification,BIRADS,Cancer Prediction,Calcification Prediction,Mass Prediction,Cancer,Calcification,CVATPAGE,tBIRAD
11,28371957-96,CLINICAL: History of benign CNB in right ...,1.0,0.0,1.0,BIRADS2,0.990935,0.784514,0.990935,-2,-2,205,BIRADS 2
18,22322096-45,CLINICAL: Follow up \nBILATERAL DIGITAL MAMMOG...,1.0,0.0,1.0,BIRADS3,0.991631,0.991631,0.804862,-2,-2,12,BIRADS 3
52,28516304-128,CLINICAL: Screening\nBILATERAL DIGITAL MAMMOG...,1.0,0.0,0.0,BIRADS4B,0.955580,0.905647,0.955580,-2,-2,413,BIRADS 4
82,22400684-201,"CLINICAL: Case right breast cancer S/P MRM, T...",1.0,0.0,1.0,BIRADS2,0.855822,0.850483,0.855822,-2,-2,15,BIRADS 2
100,22412029-49,CLINICAL: Screening with no presenting symptom...,1.0,0.0,0.0,BIRADS2,0.914696,0.914696,0.574196,-2,-2,17,BIRADS 2


In [50]:
df.to_csv('final_result.csv')

In [51]:
df[(df['Mass Prediction']>=Threshold)&(df['Calcification Prediction']<Threshold)&(df['Mass']=='1.0')]

,Image Index,Reports,Mass,Suspicious_calcification,All_calcification,BIRADS,Cancer Prediction,Calcification Prediction,Mass Prediction,Cancer,Calcification,CVATPAGE,tBIRAD
115,22436027-51,CLINICAL: Follow up study\nBILATERAL DIGITAL...,1.0,0.0,1.0,BIRADS3,0.622415,0.209827,0.622415,-2,-2,22,BIRADS 3
218,25573011-46,CLINICAL: Follow up study \nBILATERAL DIGITA...,1.0,0.0,0.0,BIRADS2,0.745814,0.080763,0.745814,-2,-2,63,BIRADS 2
286,28367395-184,BILATERAL MAMMOGRAM AND SONOGRAM\nClinical; sc...,1.0,0.0,1.0,BIRADS2,0.506000,0.020700,0.506000,-2,-2,200,BIRADS 2
336,28433564-188,Mammography and Ultrasonog...,1.0,0.0,0.0,BIRADS2,0.981198,0.002416,0.981198,-2,-2,256,BIRADS 2
348,28489952-578,Addendum Report at 06/01/2019 12:36:05\nBilate...,1.0,0.0,1.0,BIRADS3,0.958180,0.033312,0.958180,-2,-2,351,BIRADS 3
427,28548046-182,Mammogram and US breasts:\n\n\nBilateral MLO ...,1.0,0.0,1.0,BIRADS2,0.930383,0.004411,0.930383,-2,-2,471,BIRADS 2
477,28582965-588,MAMMOGRAPHY OF BOTH BREASTS: \nHistory: For ch...,1.0,0.0,1.0,BIRADS3,0.738817,0.479552,0.738817,-2,-2,520,BIRADS 3
565,28660939-540,"SCREENING BILATERAL MAMMOGRAM CC, MLO VIEWS WI...",1.0,0.0,0.0,BIRADS3,0.572493,0.009534,0.572493,-2,-2,634,BIRADS 3


#### 256,548,121